# Production

## 1. Data/settings

If the local variable $t$ has been defined already, the notebook is running from another notebook. If this is not the case, a number of "local" methods have to be carried out before solving the model (e.g. setting up the relevant IO data set):

In [1]:
try:
    t = t
except NameError:
    clean_up = True
    %run StdPackages.ipynb
    name, t = 'm', 1990
    ws = gams.GamsWorkspace(working_directory=d['work']) # specify where you want to run the GAMS models from (here the repository referred to in d['work'])
    with open(f"{d['processedData']}\\glob_{t}","rb") as file: # load global settings anmed glob_name
        glob=pickle.load(file)
    db_IO = GpyDB(pickle_path = os.path.join(d['processedData'], f'{name}_{t}')) # load IO database named IO_name
    addTimeToDb(db_IO, glob.db['t'].vals, exceptions = ['sigma'])
module = '_'.join([name,str(t),'P'])
db_IO0 = db_IO.copy() # create copy
db_IO1 = db_IO.copy() # we need an extra one here

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.
The file _gams_py_gdb13.gdx is still active and was not deleted.
The file _gams_py_gdb16.gdx is still active and was not deleted.
The file _gams_py_gdb17.gdx is still active and was not deleted.
The file _gams_py_gdb18.gdx is still active and was not deleted.
The file _gams_py_gdb3.gdx is still active and was not deleted.


Add durable prices (for now, to solve static calibration):

In [2]:
db_IO0['pD'] = db_IO0.get('pD_dur').combine_first(db_IO0.get('pD'))
db_IO0['p'] = db_IO0.get('pD_dur').groupby(['t','n']).mean().combine_first(db_IO0.get('p'))

## 2. Initial values

Set up nesting tree using the nesting structure:

In [3]:
# nest = adj.rc_pd(db_IO.get('nestProduction'), pd.Index(['1_'], name = 's')) # test model with only one of the sectors
# tree = nestingTree.tree(name = f'{module}_ces', tree = nest.to_list()) # individual tree

tree = nestingTree.tree(name = f'{module}_ces', tree = db_IO.get('nestProduction').to_list()) # individual tree
Tree = nestingTree.aggTree(name = module, trees = {tree.name: tree})(namespace = {str(n)+'_input': n for n in db_IO.get('n')})

Initialize production module, without any durables at first:

In [4]:
P = mProductionEmissions.Production(tree=Tree, glob = glob) # initialize module from nesting tree and global settings
aggregateDB.subset_db(db_IO0, Tree.db.get('s')) # goes through all symbols in db_IO and only keep the elements that are in the set 's' from Tree.db
aggregateDB.subset_db(db_IO0, Tree.get('n')) # goes through all symbols in db_IO and only keep the elements that are in the set 'n' from Tree.db
robust.robust_merge_dbs(P.s.db, db_IO0, priority = 'second') # Merge IO data into the database of the module; if a symbol is in both, prioritize records from the second database

Add value shares:

In [5]:
v = valueShares.valueShares(Tree, db_IO0)
v.compile() # set up model structure, and make sure to initialize symbols if they are not yet defined in the database 
v.write(); # write GAMS code used for the model
m = v.run(exportTo = d['work'],ws=ws) # solve the "model".

Use value shares to initialize variables:
* Outputs and inputs are provided by IO data.
* For intermediate goods, assume a price of 1 (default option in the class, so we don't have to do anything) and set value share = quantity.
* Set share parameters to the ones identified by value share system.

In [6]:
gpyDB.GpyDBs_AOM_Second(P.s.db, gpy(adj.rc_pd(m.out_db.get('vD'), P.get('int')).rename('qD'))) # set intermediate goods levels
gpyDB.GpyDBs_AOM_Second(P.s.db, gpy(m.out_db.get('mu').xs(P.get('t0')[0]).rename('mu'))) # set intermediate goods levels

### 3. Static calibration

In [7]:
P.compile(initDB=True) # set up model structure, and make sure to initialize symbols if they are not yet defined in the database (initDB = True)
P.s.setstate('C') # set to calibration state
P.write(); # write GAMS code
mStatic = P.run(exportTo = d['work'],ws=ws,**{'cns': 'CONOPT4'}) # solve the model using CONOPT4.

### 4. Dynamic calibration

Initialize module with static calibration as data:

In [8]:
P = mProductionEmissions.Production_ExoMu(tree = Tree, glob = glob)
robust.robust_merge_dbs(P.s.db, mStatic.out_db.getTypes(['variable','scalar_variable']), priority = 'second')

Add durables and remove ```MData``` (includes years that are not used in the model):

In [9]:
P.s.db['MData'] = adj.rc_pd(P.get('MData'),P.get('t'))
aggregateDB.readSets(P.s.db) # this makes sure that all necessary sets are defined in the database
P.addDurables(dur = db_IO1.get('dur_p'), dur2inv = db_IO1.get('dur2inv'), f = 'sqrAdjCosts')
P.initDurables() # adjust to steady state-like model
P.compile(initDB=True) # set up model structure
P.write(); # write GAMS code
# mBaseline = P.run(exportTo = d['work'],ws=ws,**{'cns': 'CONOPT4'}) # solve the model using CONOPT4; baselinea

Solve dynamic calibration:

In [10]:
aggregateDB.subset_db(db_IO1, P.get('t0')) # Only keep IO data on baseline year
mCalib, cp = P.sneakyCalib(db_IO1, exportTo = d['work'],ws=ws, loop_kwargs = {'n': 20}, **{'cns': 'CONOPT4'})

### 5. Export 

In [11]:
robust.robust_merge_dbs(P.s.db, mCalib.out_db.getTypes(['variable','scalar_variable']), priority = 'second')
P.s.setstate('B')

Look at durables:

In [12]:
# adj.rc_pd(P.get('qD'), P.get('dur')).groupby(['t','s']).sum().unstack('s').plot();

Export:

In [13]:
with open(os.path.join(d['modules'], P.name), "wb") as file:
    pickle.dump(P,file)